<a href="https://colab.research.google.com/github/DmitriiKnyazkin/Yolo_TEST/blob/master/MP_ver_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install mediapipe
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [19]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from google.colab.patches import cv2_imshow

In [21]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

In [22]:
cap = cv2.VideoCapture('/content/video_in_4.mp4')
imageWidth = cap.get(3)# Разрешение входного файла, ширина
imageHeight = cap.get(4)# Разрешение входного файла, высота
fps = cap.get(cv2.CAP_PROP_FPS)  # Частота входного файла, FPS
len_freame = int(cap.get(7)+1) # Длина выходного файла, кадры
output_width = 640 # Разрешение выходного файла, ширина
output_height = 480 # Разрешение выходного файла, высота

writer = cv2.VideoWriter("/content/video_out_3.avi", cv2.VideoWriter_fourcc(*"XVID"), fps, (output_width, output_height))

for i in range(len_freame):
  cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
  res, frame = cap.read()
  mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
  detection_result = detector.detect(mp_image)
  detection_result.face_landmarks[0][6].y
  detection_result.face_landmarks[0][6].x
  #Высчитываем координаты кадра относительно ценра глаз (точка 6)
  y_zenter = detection_result.face_landmarks[0][6].y * imageHeight
  x_zenter = detection_result.face_landmarks[0][6].x * imageWidth
  y_0 = int(y_zenter - output_height/2)
  y_1 = y_0 + output_height
  x_0 = int(x_zenter - output_width/2)
  x_1 = x_0 + output_width
  frame_cut_out = frame[y_0:y_1, x_0:x_1]
  writer.write(frame_cut_out)

writer.release()

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [23]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.3/873.3 kB 14.3 MB/s eta 0:00:00


In [1]:

import pandas as pd
# create a DataFrame

In [13]:
new_dict=pd.DataFrame({'x1':2, 'x2':3, 'y1' :4, 'y2':5}, index =[0])

In [16]:
#df = pd.DataFrame(data)
print(df)
df.to_csv('mydata.csv', index=False)

   x1  x2  y1  y2
0   3   5   7   9
0   2   3   4   5


In [26]:
df = pd.DataFrame({'x1':2, 'x2':3, 'y1' :4, 'y2':5}, index =[0])

In [27]:
from ultralytics import YOLO


# Загрузка модели YOLOv8
model = YOLO('/content/1280_200.pt') #YOLO('yolov8n.pt')

# Список цветов для различных классов
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (192, 192, 192), (128, 128, 128), (128, 0, 0), (128, 128, 0),
    (0, 128, 0), (128, 0, 128), (0, 128, 128), (0, 0, 128), (72, 61, 139),
    (47, 79, 79), (47, 79, 47), (0, 206, 209), (148, 0, 211), (255, 20, 147)
]

# Открытие исходного видеофайла
input_video_path = 'video_out_3.avi'
capture = cv2.VideoCapture(input_video_path)

# Чтение параметров видео
fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Настройка выходного файла
output_video_path = 'video_in_detect_3.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while True:
    # Захват кадра
    ret, frame = capture.read()
    if not ret:
        break

    # Обработка кадра с помощью модели YOLO
    results = model(frame)[0]

    # Получение данных об объектах
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy().astype(np.int32)
    # Рисование рамок и подписей на кадре
    for class_id, box, conf in zip(classes, boxes, results.boxes.conf):
        if conf>0.5:
            class_name = classes_names[int(class_id)]
            color = colors[int(class_id) % len(colors)]
            x1, y1, x2, y2 = box
            new_dict=pd.DataFrame({'x1':x1, 'x2':x2, 'y1' :y1, 'y2':y2}, index =[0])
            df = pd.concat([df, new_dict])
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                # Запись обработанного кадра в выходной файл
    writer.write(frame)
    df.to_csv('mydata3.csv', index=False)


# Освобождение ресурсов и закрытие окон
capture.release()
writer.release()


0: 960x1280 2 iriss, 2 eyes, 601.0ms
Speed: 33.1ms preprocess, 601.0ms inference, 1.5ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 2 iriss, 2 eyes, 617.1ms
Speed: 17.6ms preprocess, 617.1ms inference, 1.1ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 2 iriss, 2 eyes, 806.5ms
Speed: 18.4ms preprocess, 806.5ms inference, 1.4ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 2 iriss, 2 eyes, 896.1ms
Speed: 22.7ms preprocess, 896.1ms inference, 1.5ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 2 iriss, 2 eyes, 917.7ms
Speed: 26.2ms preprocess, 917.7ms inference, 1.6ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 2 iriss, 2 eyes, 934.6ms
Speed: 23.5ms preprocess, 934.6ms inference, 1.1ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 2 iriss, 2 eyes, 602.1ms
Speed: 17.5ms preprocess, 602.1ms inference, 1.1ms postprocess per image at shape (1, 3, 960, 1280)

0: 960x1280 2 iriss, 2 eyes, 605.